In [52]:
# Import depedencies
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score

In [22]:
!pip install xgboost

In [28]:
# Read the CSV data and create a new dataframe

df = pd.read_csv("match_team_data.csv")

In [29]:
# Create a column to account for Wins, Losses, and Ties
# Win = 1 , Loss = 0, Tie = -1

#We changed 1 to win and 0 to lose because XGBoost only accepts binary values.

df["home_win"] = np.where(df["home_score"] > df["away_score"], 1,0)

In [30]:
# Check for NaNs
df.isna().mean().sort_values(ascending=False).head(10)

team_2_post_season_result                1.000000
team_1_post_season_result                1.000000
winning_abbr                             0.079338
winning_name                             0.079338
losing_name                              0.079338
losing_abbr                              0.079338
away_score                               0.075625
home_score                               0.075625
team_2_pass_completions                  0.000000
team_2_offensive_simple_rating_system    0.000000
dtype: float64

In [31]:
df["winning_name"].isna()

0       False
1       False
2       False
3       False
4       False
        ...  
2957     True
2958     True
2959     True
2960     True
2961     True
Name: winning_name, Length: 2962, dtype: bool

In [32]:
# Remove NaNs
df = df.drop(columns=["team_2_post_season_result","team_1_post_season_result"])

In [33]:
df3 = df.dropna()

In [34]:
df3

boxscore            away_name away_abbr  away_score  \
0     201209050nyg       Dallas Cowboys       dal        24.0   
1     201209090chi   Indianapolis Colts       clt        21.0   
2     201209090cle  Philadelphia Eagles       phi        17.0   
3     201209090det       St. Louis Rams       ram        23.0   
4     201209090htx       Miami Dolphins       mia        10.0   
...            ...                  ...       ...         ...   
2733  202209250crd     Los Angeles Rams       ram        20.0   
2734  202209250sea      Atlanta Falcons       atl        27.0   
2735  202209250tam    Green Bay Packers       gnb        14.0   
2736  202209250den  San Francisco 49ers       sfo        10.0   
2737  202209260nyg       Dallas Cowboys       dal        23.0   

                 home_name home_abbr  home_score         winning_name  \
0          New York Giants       nyg        17.0       Dallas Cowboys   
1            Chicago Bears       chi        41.0        Chicago Bears   
2         Cleveland Browns       cle        16.0  Philadelphia Eagles   
3            Detroit Lions       det        27.0        Detroit Lions   
4           Houston Texans       htx        30.0       Houston Texans   
...                    ...       ...         ...                  ...   
2733     Arizona Cardinals       crd        12.0     Los Angeles Rams   
2734      Seattle Seahawks       sea        23.0      Atlanta Falcons   
2735  Tampa Bay Buccaneers       tam        12.0    Green Bay Packers   
2736        Denver Broncos       den        11.0       Denver Broncos   
2737       New York Giants       nyg        16.0       Dallas Cowboys   

     winning_abbr           losing_name  ... team_2_rush_yards_per_attempt  \
0             dal       New York Giants  ...                           4.7   
1             chi    Indianapolis Colts  ...                           4.0   
2             phi      Cleveland Browns  ...                           4.4   
3             det        St. Louis Rams  ...                           3.4   
4             htx        Miami Dolphins  ...                           3.3   
...           ...                   ...  ...                           ...   
2733          ram     Arizona Cardinals  ...                           3.4   
2734          atl      Seattle Seahawks  ...                           4.9   
2735          gnb  Tampa Bay Buccaneers  ...                           4.7   
2736          den   San Francisco 49ers  ...                           4.5   
2737          dal       New York Giants  ...                           4.7   

      team_2_simple_rating_system  team_2_strength_of_schedule  \
0                            -4.2                         -2.2   
1                            -4.8                          2.2   
2                            38.9                         26.9   
3                            -9.0                         -6.0   
4                            12.4                          6.1   
...                           ...                          ...   
2733                         -9.0                         -6.0   
2734                        -11.3                        -10.9   
2735                         12.5                         11.5   
2736                         -4.8                         -8.2   
2737                         -4.2                         -2.2   

     team_2_turnovers team_2_win_percentage  team_2_wins  team_2_yards  \
0                   2                 0.667            2           972   
1                   6                 0.500            1           994   
2                   1                 1.000            3          1341   
3                   7                 0.667            2           919   
4                   2                 1.000            3          1066   
...               ...                   ...          ...           ...   
2733                7                 0.667            2           919   
2734                7         

In [35]:
df3["home_win"].value_counts()


1    1520
0    1207
Name: home_win, dtype: int64

In [36]:
# Check for categorical data

df3.dtypes.sort_values().tail(15)

team_1_points_contributed_by_offense    float64
away_name                                object
away_abbr                                object
home_name                                object
home_abbr                                object
team_2_abbreviation                      object
losing_name                              object
winning_abbr                             object
losing_abbr                              object
teams                                    object
team_1_abbreviation                      object
team_2_name                              object
team_1_name                              object
winning_name                             object
boxscore                                 object
dtype: object

In [37]:
# Remove categorical data

df3.drop(columns=["away_name","away_abbr","home_name","home_abbr","team_2_abbreviation","losing_name","winning_abbr","losing_abbr","teams","team_1_abbreviation","team_2_name","team_1_name","winning_name","boxscore"], inplace=True)

C:\Users\yohan\anaconda3\envs\dev\envs\dev\lib\site-packages\pandas\core\frame.py:4913: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [38]:
# Remove categorical data

df3.drop(columns=["year","week","home_score","away_score"], inplace=True)

In [39]:
# Create target and features

y = df3["home_win"]
X = df3.drop(columns="home_win")

In [40]:
# Apply Train_Test_Split

X_train, X_test, y_train, y_test = train_test_split(X,y,random_state=1)

In [41]:
# Set the scaler
scaler = StandardScaler()

In [42]:
# Fit and Transform the data
X_scaler = scaler.fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [43]:
from xgboost import XGBClassifier

In [44]:
# Create the model
model = XGBClassifier()

In [45]:
# Fit the data
model.fit(X_train,y_train)

XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',
              importance_type=None, interaction_constraints='',
              learning_rate=0.300000012, max_bin=256, max_cat_to_onehot=4,
              max_delta_step=0, max_depth=6, max_leaves=0, min_child_weight=1,
              missing=nan, monotone_constraints='()', n_estimators=100,
              n_jobs=0, num_parallel_tree=1, predictor='auto', random_state=0,
              reg_alpha=0, reg_lambda=1, ...)

In [46]:
# Make the predictions
y_pred = model.predict(X_test)

In [53]:
# print the results

print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))
print(accuracy_score(y_test, y_pred))

[[141 157]
 [156 228]]
              precision    recall  f1-score   support

           0       0.47      0.47      0.47       298
           1       0.59      0.59      0.59       384

    accuracy                           0.54       682
   macro avg       0.53      0.53      0.53       682
weighted avg       0.54      0.54      0.54       682

0.5410557184750733


In [1]:
pwd

'C:\\Users\\yohan\\nuBootCamp\\Project-2-Team-1'